# Extrator de informações genérico

In [3]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import re

## Criando lista de páginas

In [51]:
soups = []
files = []
for dirpath,_,filenames in os.walk(os.path.abspath('..\\Pages\\Positives\\')):
    for f in filenames:
        file = os.path.abspath(os.path.join(dirpath, f))
        files.append(file)
        soups.append(BeautifulSoup(
        open(file, encoding='utf-8'), 'html.parser'))
print(len(files))
len(soups)

91


91

## Extraindo nomes dos psicologos

Antes de começarmos vamos examinar rapidamente os campos h1's das páginas, uma vez que imaginasse que o nome do psicólogo deva ser encontrado neste campo

In [64]:
def clean_text(text):
    return text.replace('\n', '').replace('\t', '');

for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        print(str(index) + ' - ' + str(len(h1s)) + ' -- ' + clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        print(str(index) + ' - ' + str(len(h1s)) + ' - ' + str(h1s[0].parent.name) +
              ' - ' + clean_text(h1s[0].text) + ' -- ' + str(h1s[1].parent.name) + ' - '+ clean_text(h1s[1].text))
    else:
        print( str(index) + ' - Empty result')



0 - Empty result
1 - Empty result
2 - Empty result
3 - Empty result
4 - Empty result
5 - Empty result
6 - Empty result
7 - Empty result
8 - Empty result
9 - Empty result
10 - 2 - div - Sérgio Alexandre Alves Fernandes -- div - Sérgio Alexandre Alves Fernandes
11 - 2 - div - Luiz Gustavo Canuto -- div - Luiz Gustavo Canuto
12 - 2 - div - Igor Costa -- div - Igor Costa
13 - 2 - div - Bruno Schievenin Madi -- div - Bruno Schievenin Madi
14 - 2 - div - Sandro Azevêdo -- div - Sandro Azevêdo
15 - 2 - div - Wilza M Brosig -- div - Wilza M Brosig
16 - 2 - div - Dra. Luciana Dias Nascimento -- div - Dra. Luciana Dias Nascimento
17 - 2 - div - Fernanda Simionato Costa -- div - Fernanda Simionato Costa
18 - 2 - div - Rodrigo Noia Mattos Montan -- div - Rodrigo Noia Mattos Montan
19 - 2 - div - Kassia Arouca cardoso -- div - Kassia Arouca cardoso
20 - 2 - div - Dr. Caio Fasoli Rebouças -- div - Dr. Caio Fasoli Rebouças
21 - 1 -- ADRIANA PANZAN
22 - 1 -- CONSULTORIO DE PSICOLOGIA SORAIA S GRANADO 

Analisando a coleta acima é possível perceber que existem poucos padrões diferentes.

Primeiro podemos identificar que as páginas possuem de 0 a 2 campos com a tag H1.

Onde o classificados não possui uma tag h1 em seu corpo, enquanto o doctoralia e o telelistas possuem essa tag 2x em seu corpo, além disto o dominio <b>psicolink</b> possui 1 página outlier que possui 2 tags H1 na mesma página, enquanto todas as demais possuem apenas 1.

A maioria dos dominios pode se utilizar apenas pegar o primeiro campo H1 para se extrair o campo nome, porém os dominios Psicólgoos do Brasil, telelistas e classificados precisam de um tratamento extra para serem cobertos.

Segue abaixo suas particularidades.
O dominio <b>classificados</b> não possui a tag h1, onde é possível observar que seus campos estão sempre dentro de uma tabela, talvez esse domínio seja o mais diferente dentre os utilizados.

O <b>telelistas</b> é o único cuja tag <b>H1</b> esta dentro de uma tag <b>A</b> e esta deve ser ignorada, também sendo o único em que o campo nome não esta na primeira tag <b>H1</b> encontrada e ainda apesar do campo nome se encontrar na segunda tag <b>H1</b> o mesmo vem com a cidade pós-fixada dentro do próprio campo.

O domínio <b>psicólogos do brasil</b> possui o campo nome na tag <b>H1</b>, porém o mesmo trás como prefixo o nome do próprio domínio.

Assim com o algoritmo de extrair o campo diretamente da primeira tag <b>H1</b> encontrada, possuimos:

<b>Cobertura: 89%</b>
<b>Precisão: 75.3%</b>

### Melhorando a extração dos nomes
Desta forma a primeira estratégia que se vem a mente para aumentar a precisão do extrator genérico é adicionar condicionais a cerca da seleção da tag quando encontrarmos mais que uma tag <b>H1</b> na página, ao simplesmente alterar a estratégia para pegar a última tag <b>H1</b>, obtemos uma <b>Precisão de 86.4%</b> pois coletaremos corretamente todas as páginas do domínio <b>Telelistas</b>, porém o outlier encontrado no domínio <b>psicolink</b> não será extraido corretamente.